# Laboratorio 7: Red neuronal de clasificación binaria

Estudiante: Alejandra Arciniegas Marin, C.C 1000662159

In [29]:
#@title Importar librerías

import numpy as np
import h5py
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [30]:
#@title Definición de la clase para definir la red neuronal:

class Red_neuronal():
    '''
    Clase que representa una red neuronal para clasificación binaria.

    Parámetros del constructor:
    ----------
    L: int
        Número de capas de la red.
    topology : list of int
        Lista con el número de neuronas que tendrá cada capa.
    activation : list of str
        Funciones de activación a utilizar en cada capa.
    N_epochs : int
        Número de épocas para entrenar el algoritmo.
    N_train : int
        Número de datos de entrenamiento.
    N_test : int
        Número de datos de prueba.
    Xtrain : numpy.ndarray
        Arreglo con los datos de entrenamiento. Las filas deben ser las características y las columnas los datos.
    Ytrain : numpy.ndarray
        Arreglo con las etiquetas de los datos de entrenamiento.
    Xtest : numpy.ndarray
        Arreglo con los datos de prueba. Las filas deben ser las características y las columnas los datos.
    Ytest : numpy.ndarray
        Arreglo con las etiquetas de los datos de prueba.
    alpha : float
        Tasa de aprendizaje del gradiente descendente.
    '''

    def __init__(self, L, topology, activation, N_epochs, N_train, N_test, Xtrain, Ytrain, Xtest, Ytest, alpha):
        self.L = L
        self.topology = topology
        self.activation = activation
        self.N_epochs = N_epochs
        self.m = N_train
        self.x = Xtrain
        self.y = Ytrain
        self.N_test = N_test
        self.Xtest = Xtest
        self.Ytest = Ytest
        self.alpha = alpha
        self.weights = []
        self.biases = []

        # Inicialización de pesos y biases
        self.Initialization()

    def activation_function(self, name, x):
        '''
        Calcula la función de activación para una capa.

        Parámetros:
        ----------
        name : str
            El nombre de la función de activación ('sigmoid', 'relu', 'tanh').
        x : numpy.ndarray
            El valor de entrada.

        Retorno:
        ----------
        numpy.ndarray
            El valor de la función de activación aplicada a x.
        '''
        if name == 'sigmoid':
            return 1 / (1 + np.exp(-x))
        elif name == 'relu':
            return np.maximum(0, x)
        elif name == 'tanh':
            return np.tanh(x)

    def der_activation_function(self, name, x):
        '''
        Calcula la derivada de la función de activación para una capa.

        Parámetros:
        ----------
        name : str
            El nombre de la función de activación ('sigmoid', 'relu', 'tanh').
        x : numpy.ndarray
            El valor de entrada.

        Retorno:
        ----------
        numpy.ndarray
            El valor de la derivada de la función de activación aplicada a x.
        '''
        if name == 'sigmoid':
            sigmoid = self.activation_function('sigmoid', x)
            return sigmoid * (1 - sigmoid)
        elif name == 'relu':
            return np.where(x > 0, 1, 0)
        elif name == 'tanh':
            return 1 - np.tanh(x) ** 2

    def Forward_Pass(self, w_l, a_l_1, b_l, f):
        '''
        Realiza el paso hacia adelante de una capa en la red.

        Parámetros:
        ----------
        w_l : numpy.ndarray
            Los pesos de la capa actual.
        a_l_1 : numpy.ndarray
            La activación de la capa anterior.
        b_l : numpy.ndarray
            Los sesgos de la capa actual.
        f : str
            La función de activación de la capa actual.

        Retorno:
        ----------
        a_l : numpy.ndarray
            La activación de la capa actual.
        z_l : numpy.ndarray
            El valor antes de la función de activación.
        '''
        z_l = np.dot(w_l, a_l_1) + b_l
        a_l = self.activation_function(f, z_l)
        return a_l, z_l

    def Cost_func(self, a_l):
        '''
        Calcula la función de costo.

        Parámetros:
        ----------
        a_l : numpy.ndarray
            La activación de la última capa.

        Retorno:
        ----------
        float
            El valor de la función de costo.
        '''
        return (-1 / self.m) * np.sum(self.y * np.log(a_l) + (1 - self.y) * np.log(1 - a_l))

    def Backward_Pass_out(self, a_L, a_L_1, W_L, b_L, Y):
        '''
        Realiza el paso hacia atrás para la última capa en la red.

        Parámetros:
        ----------
        a_L : numpy.ndarray
            La activación de la última capa.
        a_L_1 : numpy.ndarray
            La activación de la penúltima capa.
        W_L : numpy.ndarray
            Los pesos de la última capa.
        b_L : numpy.ndarray
            Los sesgos de la última capa.
        Y : numpy.ndarray
            Las etiquetas de los datos.

        Retorno:
        ----------
        W_L : numpy.ndarray
            Los pesos actualizados de la última capa.
        b_L : numpy.ndarray
            Los sesgos actualizados de la última capa.
        '''
        delta_L = a_L - Y
        dJdW = (1 / self.m) * np.dot(delta_L, a_L_1.T)
        dJdb = (1 / self.m) * np.sum(delta_L, axis=1, keepdims=True)

        W_L -= self.alpha * dJdW
        b_L -= self.alpha * dJdb

        return W_L, b_L

    def Backward_Pass(self, name, delta_l1, W_l1, W_l, Z_l, a_l_1, b_l):
        '''
        Realiza el paso hacia atrás para una capa intermedia en la red.

        Parámetros:
        ----------
        name : str
            El nombre de la función de activación de la capa actual.
        delta_l1 : numpy.ndarray
            El error de la capa siguiente.
        W_l1 : numpy.ndarray
            Los pesos de la capa siguiente.
        W_l : numpy.ndarray
            Los pesos de la capa actual.
        Z_l : numpy.ndarray
            El valor antes de la función de activación de la capa actual.
        a_l_1 : numpy.ndarray
            La activación de la capa anterior.
        b_l : numpy.ndarray
            Los sesgos de la capa actual.

        Retorno:
        ----------
        W_l : numpy.ndarray
            Los pesos actualizados de la capa actual.
        b_l : numpy.ndarray
            Los sesgos actualizados de la capa actual.
        delta_l : numpy.ndarray
            El error de la capa actual.
        '''
        delta_l = np.dot(W_l1.T, delta_l1) * self.der_activation_function(name, Z_l)
        dJdW = (1 / self.m) * np.dot(delta_l, a_l_1.T)
        dJdb = (1 / self.m) * np.sum(delta_l, axis=1, keepdims=True)

        W_l -= self.alpha * dJdW
        b_l -= self.alpha * dJdb

        return W_l, b_l, delta_l

    def Initialization(self):
        '''
        Inicializa los pesos y sesgos de la red.

        Parámetros: No tiene.

        Retorno: No tiene.
        '''
        np.random.seed(1)
        self.weights = [np.random.randn(self.topology[i+1], self.topology[i]) * 0.01 for i in range(self.L-1)]
        self.biases = [np.zeros((self.topology[i+1], 1)) for i in range(self.L-1)]

    def train(self):
        '''
        Entrena la red neuronal utilizando propagación hacia adelante y hacia atrás.

        Parámetros: No tiene.

        Retorno: No tiene.
        '''
        for epoch in range(self.N_epochs):
            a = [self.x]
            z = []

            # Forward Pass
            for i in range(self.L-1):
                a_l, z_l = self.Forward_Pass(self.weights[i], a[-1], self.biases[i], self.activation[i])
                a.append(a_l)
                z.append(z_l)

            # Cálculo del costo
            cost = self.Cost_func(a[-1])

            # Backward Pass
            delta_l = a[-1] - self.y
            for i in reversed(range(self.L-1)):
                if i == self.L-2:
                    self.weights[i], self.biases[i] = self.Backward_Pass_out(a[-1], a[-2], self.weights[i], self.biases[i], self.y)
                else:
                    self.weights[i], self.biases[i], delta_l = self.Backward_Pass(self.activation[i], delta_l, self.weights[i+1], self.weights[i], z[i], a[i], self.biases[i])

    def predict(self, X):
        '''
        Realiza una predicción utilizando la red neuronal entrenada.

        Parámetros:
        ----------
        X : numpy.ndarray
            Los datos de entrada para realizar la predicción.

        Retorno:
        ----------
        numpy.ndarray
            La predicción de la red neuronal.
        '''
        a = X  # Inicialización de la activación con los datos de entrada
        for i in range(self.L-1):
            # Realizar el paso hacia adelante para cada capa
            a, _ = self.Forward_Pass(self.weights[i], a, self.biases[i], self.activation[i])
        return a  # Retornar la activación de la última capa (predicción)

    def score(self, X, Y):
        '''
        Calcula la exactitud del modelo en los datos proporcionados.

        Parámetros:
        ----------
        X : numpy.ndarray
            Los datos de entrada para realizar la predicción.
        Y : numpy.ndarray
            Las etiquetas reales de los datos de entrada.

        Retorno:
        ----------
        float
            La exactitud del modelo, como un valor entre 0 y 1.
        '''
        predictions = self.predict(X)  # Realizar predicciones en los datos de entrada
        predictions = (predictions > 0.5).astype(int)  # Convertir las probabilidades a etiquetas binarias
        return np.mean(predictions == Y)  # Calcular y retornar la exactitud de las predicciones


In [31]:
#@title Ejecución con el dataset de fotos de gatos

# Cargar el dataset
data_train = "train_catvnoncat.h5"
train_dataset = h5py.File(data_train, "r")

data_test = "test_catvnoncat.h5"
test_dataset = h5py.File(data_test, "r")

xtrain_classes, xtrain, train_label =\
train_dataset["list_classes"], train_dataset["train_set_x"], train_dataset["train_set_y"]

test_classes, xtest, test_label =\
test_dataset["list_classes"], test_dataset["test_set_x"], test_dataset["test_set_y"]

ytrain = np.array(train_label)
Xtrain = (np.reshape(xtrain, (xtrain.shape[0], -1)) / 255).T

ytest = np.array(test_label)
Xtest = (np.reshape(xtest, (xtest.shape[0], -1)) / 255).T

# Parámetros de la red
L = 4  # Número de capas
topology = [Xtrain.shape[0], 10, 10, 1]  # Topología de la red
activation = ['relu', 'sigmoid', 'sigmoid', 'sigmoid']  # Funciones de activación
N_epochs = 1000  # Número de épocas
N_train = Xtrain.shape[1]  # Número de datos de entrenamiento
N_test = Xtest.shape[1]  # Número de datos de prueba
alpha = 0.01  # Tasa de aprendizaje

# Crear y entrenar el modelo
model = Red_neuronal(L, topology, activation, N_epochs, N_train, N_test, Xtrain, ytrain, Xtest, ytest, alpha)
model.train()

# Evaluar el modelo
train_score = model.score(Xtrain, ytrain)
test_score = model.score(Xtest, ytest)
print(f'Train Score: {train_score}')
print(f'Test Score: {test_score}')


Train Score: 0.6555023923444976
Test Score: 0.34


Poe los puntajes obtenidos, se puede ver que el puntaje de entrenamiento y el de prueba están muy alejados el uno del otro, por tanto hay un overffiting del modelo. De igual forma ambos puntajes son bastante bajitos.

In [32]:
#@title Ejecución con el dataset del cáncer de mama

# Cargar el dataset
data = load_breast_cancer()
X = data.data
Y = data.target.reshape(-1, 1)

# Dividir en datos de entrenamiento y prueba
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=42)

# Estandarizar los datos
scaler = StandardScaler()
Xtrain = scaler.fit_transform(Xtrain)
Xtest = scaler.transform(Xtest)

# Convertir a las formas requeridas (transponer)
Xtrain = Xtrain.T
Ytrain = Ytrain.T
Xtest = Xtest.T
Ytest = Ytest.T

# Parámetros de la red
L = 4  # Número de capas
topology = [Xtrain.shape[0],10,5, 3, 1]  # Topología de la red (ajustada)
activation = ['relu', 'sigmoid','sigmoid', 'sigmoid']  # Funciones de activación
N_epochs = 1000  # Número de épocas
N_train = Xtrain.shape[1]  # Número de datos de entrenamiento
N_test = Xtest.shape[1]  # Número de datos de prueba
alpha = 0.01  # Tasa de aprendizaje

# Creación y entrenamiento del modelo
model = Red_neuronal(L, topology, activation, N_epochs, N_train, N_test, Xtrain, Ytrain, Xtest, Ytest, alpha)
model.train()

# Evaluación del modelo
train_score = model.score(Xtrain, Ytrain)
test_score = model.score(Xtest, Ytest)
print(f'Train Score: {train_score}')
print(f'Test Score: {test_score}')


Train Score: 0.6285714285714286
Test Score: 0.6228070175438597


Aquí, ambos puntajes son mucho más cercanos el uno del otro, sin embargo siguen siendo bastante lejanos a uno, lo que muestra un mal comportamiento del modelo. En general, el modelo no es muy bueno.